In [1]:
# Find RL_Note path and append sys path
import os, sys
cwd = os.getcwd()
pos = cwd.find('RL_Note')
root_path = cwd[0:pos] + 'RL_Note'
sys.path.append(root_path)

In [2]:
# Refer from
#  https://pasus.tistory.com/138
#  https://horomary.hatenablog.com/entry/2020/06/26/003806
#  https://keras.io/examples/rl/ddpg_LunaerLander/
#  https://github.com/dongminlee94/Samsung-DRL-Code/blob/master/5_SAC/sac/model.py
import gym
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, concatenate, Lambda
import tensorflow_probability as tfp
tfd = tfp.distributions
# import matplotlib.pyplot as plt
# from pys.utils.er import ReplayMemory

In [3]:
class Actor(tf.keras.Model):
    def __init__(self, state_size, action_size, log_std_min, log_std_max):
        super(Actor, self).__init__()
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max

        self.fc1= Dense(64, activation='relu')
        self.fc2= Dense(64, activation='relu')
        # self.fc3 = Dense(16, activation='relu')
        self.mu = Dense(action_size)
        self.log_std= Dense(action_size)

    def call(self, x):
        x       = self.fc1(x)
        x       = self.fc2(x)
        # x       = self.fc3(x)
        mu = self.mu(x)
        log_std = self.log_std(x)
        log_std = tf.clip_by_value(log_std, self.log_std_min, self.log_std_max)
        std = tf.math.exp(log_std)
        return mu, std

class Critic(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.s1 = Dense(16, activation='relu')
        self.s2 = Dense(32, activation='relu')
        self.a1 = Dense(32, activation='relu')
        self.a2 = Dense(32, activation='relu')
        self.fc1= Dense(64, activation='relu')
        self.fc2= Dense(64, activation='relu')
        self.out= Dense(1,  activation='linear')

    def call(self,state,action):
        # state  = state_action[0]
        # action = state_action[1]
        s = self.s1(state)
        s = self.s2(s)
        a = self.a1(action)
        a = self.a2(a)
        c = concatenate([s,a],axis=-1)
        x = self.fc1(c)
        x = self.fc2(x)
        q = self.out(x)
        return q

In [4]:
# https://github.com/dongminlee94/Samsung-DRL-Code/blob/master/5_SAC/sac/utils.py
# https://github.com/RickyMexx/SAC-tf2/blob/master/SAC/SAC_rla.py
# https://github.com/p-christ/Deep-Reinforcement-Learning-Algorithms-with-PyTorch/blob/b338c87bebb672e39304e47e0eed55aeb462b243/agents/Base_Agent.py#L278
# 
class SACAgent:
    def __init__(self, state_size, action_size, action_min, action_max):
        self.state_size = state_size
        self.action_size= action_size
        self.action_min = action_min
        self.action_max = action_max

        self.actor          = Actor(self.state_size, self.action_size, self.action_min, self.action_max)
        self.actor.build(input_shape=(None, self.state_size))
        state_in = Input(shape=(self.state_size,),dtype=tf.float32)
        self.actor(state_in)
        self.actor.summary()
        self.load_model()
        
    def get_action(self, state):
        state = tf.convert_to_tensor([state], dtype=tf.float32)
        mu, std = self.actor(state)
        action, _ = self.eval_action(mu,std)
        return action.numpy()[0]

    def eval_action(self, mu, std, epsilon=1e-6):
        action_prob = tfd.Normal(loc=mu, scale=std)
        z = action_prob.sample()
        action = tf.math.tanh(z)
        log_prob = action_prob.log_prob(z) - tf.math.log(1.0 - tf.pow(action,2) + epsilon)
        log_prob = tf.reduce_sum(log_prob, axis=-1, keepdims=True)

        return action, log_prob

    def load_model(self):
        self.actor.load_weights(  "./save_model/LunarLanderC_sac_fixed_TF_actor")
        # self.critic1.load_weights("./save_model/LunarLanderC_sac_fixed_TF_critic1", save_format="tf")
        # self.critic2.load_weights("./save_model/LunarLanderC_sac_fixed_TF_critic2", save_format="tf")
        return


In [5]:
%matplotlib tk

ENV_NAME = 'LunarLanderContinuous-v2'
EPISODES = 10
END_SCORE = 200

if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    state_size      = env.observation_space.shape[0]
    action_size     = env.action_space.shape[0]
    log_std_min     = -20.0
    log_std_max     = 5.0

    agent = SACAgent(state_size, action_size, log_std_min, log_std_max)
    print('Env Name : ',ENV_NAME)
    print('States {0}, Actions {1}'.format(state_size, action_size))
    print('Action scale exp({0:.2f} ~ {1:.2f})'.format(log_std_min, log_std_max))
    score_avg = 0

    end = False
    show_media_info = True

    for e in range(EPISODES):
        done = False
        score = 0
        state = env.reset()
        while not done:
            # if e%100 == 0:
            env.render()
            # Interact with env.
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            state = next_state
            # 
            score += reward
            if show_media_info:
                print("-------------- Variable shapes --------------")
                print("State Shape : ", np.shape(state))
                print("Action Shape : ", np.shape(action))
                print("Reward Shape : ", np.shape(reward))
                print("done Shape : ", np.shape(done))
                print("---------------------------------------------")
                show_media_info = False
            if done:
                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print("episode: {0:3d} | score avg: {1:3.2f} |"
                    .format(e, score_avg))
                if e==EPISODES:
                    end = True
        if end:
            break
    env.close()
            

Model: "actor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  576       
_________________________________________________________________
dense_1 (Dense)              multiple                  4160      
_________________________________________________________________
dense_2 (Dense)              multiple                  130       
_________________________________________________________________
dense_3 (Dense)              multiple                  130       
Total params: 4,996
Trainable params: 4,996
Non-trainable params: 0
_________________________________________________________________
Env Name :  LunarLanderContinuous-v2
States 8, Actions 2
Action scale exp(-20.00 ~ 5.00)
-------------- Variable shapes --------------
State Shape :  (8,)
Action Shape :  (2,)
Reward Shape :  ()
done Shape :  ()
---------------------------------------------

C:\Users\user\anaconda3\envs\tf241\lib\site-packages\ipykernel\eventloops.py:256: RuntimeWarning: coroutine 'Kernel.do_one_iteration' was never awaited
  self.func()
